In [1]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}

from datetime import datetime
import json 
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    #Not recursive 
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    
    changes_dict = {'TermType':'CT'}
    
    copied_json = {}
    stack = [(copied_json, context)]

    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value

    # Tüm değerlerin tarihlerini ayarla
    def set_dates(data_dict):
        if isinstance(data_dict,dict):
            if data_dict.get('value') is None and 'values' in data_dict and data_dict['values']:
                data_dict['values'][0]['startDate'] = current_date
                data_dict['values'][0]['endDate'] = 99991231235959
            elif data_dict.get('value') is not None:
                data_dict['startDate'] = current_date
                data_dict['endDate'] = 99991231235959
    
    def update_attribute_value(context, update_dict):

        for code, new_value in update_dict.items():
            
            for attr in context['attributes']:
                if attr['attribute']['attributeCode'] == code:
                    if attr['value'] is None:
                        attr['values'][0]['value'] = new_value
                        attr['values'][0]['valueDesc'] = None
                    # Yeni değeri güncelle
                    else:
                        attr['values'][0]['valueDesc'] = None
                        attr['value'] = new_value
                    break
        return context
    
    update_attribute_value(copied_json,changes_dict)
            
    for key, value in copied_json.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    set_dates(item)
        elif isinstance(value, dict):
            set_dates(value)

    return copied_json   

In [8]:
with open(r'C:\Users\KaanGermiyan\StateMachine-2\Double_Term_Single_Term\inputs\state1_input.json') as f:
    context = json.load(f)


{'id': None,
 'attribute': {'alias': None, 'attributeCode': 'TermType'},
 'value': None,
 'startDate': None,
 'endDate': None,
 'valueDesc': None,
 'values': [{'id': 13622,
   'attribute': None,
   'value': 'TT',
   'startDate': 20241001000000,
   'endDate': 99991231235959,
   'valueDesc': 'Tek Terim',
   'values': None}]}

In [9]:
update_dict = {'TermType':'CT'}

In [13]:
def update_attribute_value(context, update_dict):

    for code, new_value in update_dict.items():
        
        for attr in context['attributes']:
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['endDate'] = current_date
                    attr['values'][0]['value'] = new_value
                # Yeni değeri güncelle
                else:
                    attr['endDate'] = current_date
                    attr['value'] = new_value
                break
    return context


In [11]:
context = update_attribute_value(context,update_dict)

CT


In [12]:
response =  next((attr for attr in context['attributes'] if attr['attribute']['attributeCode']=='TermType'),None)

In [39]:
response

{'id': None,
 'attribute': {'alias': None, 'attributeCode': 'TermType'},
 'value': None,
 'startDate': None,
 'endDate': None,
 'valueDesc': None,
 'values': [{'id': 13622,
   'attribute': None,
   'value': 'CT',
   'startDate': 20241001000000,
   'endDate': 99991231235959,
   'valueDesc': 'Tek Terim',
   'values': None}]}

In [14]:
update_dict = {'TermType':'CT',
               'valueDesc' : 'Çift Terim'}

In [20]:
update_dict['TermType']

'CT'